# 🧑‍🎓 Assignment: Orthogonalization in ML (with penalty & JSON output)
Заполни **Student Info**, установи дедлайны, выполни задания. В конце — вывод баллов и финальный JSON одной строкой.


Fill in **Student Info**, set the deadlines, complete the tasks. At the end — output the scores and the final JSON in a single line.

## Student Info

In [ ]:

# === ОБЯЗАТЕЛЬНО ЗАПОЛНИТЬ ===
full_name = "Фамилия Имя"     # например: "Тощев Александр"
student_group = "Группа"      # например: "208"
assignment_id = "ORTHO-01"
assert full_name != "Фамилия Имя", "Заполните full_name"
assert student_group != "Группа", "Заполните student_group"
print("✔ Student Info OK")


## Deadlines & Penalty

In [ ]:

from datetime import datetime, timezone, timedelta

# Установите окна приёма (пример):
start_dt = datetime(2025, 9, 1, 0, 0, 0, tzinfo=timezone.utc)
due_dt   = datetime(2025, 10, 1, 23, 59, 59, tzinfo=timezone.utc)

# Для протокола: время сдачи берём текущее (можно заменить на mtime файла)
submission_dt = datetime.now(timezone.utc)

def penalty_fraction(start_dt, due_dt, submission_dt):
    """Возвращает долю штрафа [0..1].
    0 — без штрафа (<= due_dt). Линейно растёт от due_dt к due_dt + (due_dt - start_dt).
    Не выходит за 1.0.
    """
    if submission_dt <= due_dt:
        return 0.0
    total = (due_dt - start_dt).total_seconds()
    late  = (submission_dt - due_dt).total_seconds()
    if total <= 0:
        return 1.0 if late > 0 else 0.0
    return min(1.0, max(0.0, late / total))

print(f"Окно приёма: {start_dt.isoformat()} — {due_dt.isoformat()} (UTC)")
print(f"Время сдачи: {submission_dt.isoformat()} (UTC)")


## Task 1 — Грам–Шмидт (30 баллов) Gram Shmidt


In [ ]:

import numpy as np
raw_score = 0
max_points = 100

def my_gram_schmidt(V: np.ndarray) -> np.ndarray:
    U = []
    for v in V.astype(float):
        for u in U:
            v = v - np.dot(v, u) * u
        n = np.linalg.norm(v)
        if n > 1e-12:
            U.append(v / n)
    return np.array(U)

# [TEST]
V = np.array([[1, 1, 0], [1, 0, 1], [2, 1, 1]], dtype=float)
U = my_gram_schmidt(V)
assert U.shape[0] >= 2, "Ожидаем минимум 2 вектора 2 vectors awaiting"
# ортогональность
for i in range(len(U)):
    for j in range(i+1, len(U)):
        assert abs(np.dot(U[i], U[j])) < 1e-6, "Неортогональные векторы, Non-orthogonal vectors"
# нормы
assert np.allclose(np.linalg.norm(U, axis=1), 1, atol=1e-6), "Векторы не нормированы, vectors not normalized"
raw_score += 30
print("✔ Task 1 passed: +30")


## Task 2 — PCA на Iris (30 баллов)

In [ ]:

import pandas as pd
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA

iris = load_iris()
X = iris.data
y = iris.target

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
VAR2 = float(pca.explained_variance_ratio_.sum())

assert 0.6 <= VAR2 <= 1.0, f"Слишком низкая суммарная дисперсия, Dispersation too low: {VAR2}"
raw_score += 30
print("✔ Task 2 passed: +30")


## Task 3 — Ridge (40 баллов)

In [ ]:

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X2 = X[:, :2]
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=42)
ALPHAS = [0.001, 0.01, 0.1, 1, 10, 100]
mses = []
for a in ALPHAS:
    m = Ridge(alpha=a)
    m.fit(X_train, y_train)
    pred = m.predict(X_test)
    mses.append(mean_squared_error(y_test, pred))

BEST_MSE = float(min(mses))
BEST_ALPHA = float(ALPHAS[int(np.argmin(mses))])
assert BEST_MSE < 1.5, f"Подозрительно большая MSE, very hogh MSE: {BEST_MSE}"
raw_score += 40
print("✔ Task 3 passed: +40")


## Итог и JSON (совместимо с вашим harness)

In [ ]:

import json

# применяем штраф
try:
    pf = penalty_fraction(start_dt, due_dt, submission_dt)
except NameError:
    from datetime import timezone
    pf = 0.0

final_score = max(0.0, raw_score * (1.0 - min(1.0, pf)))

print(f"Сырой балл: {raw_score}/{max_points}")
print(f"Штраф (доля): {pf:.4f}")
print(f"Итоговый балл после штрафа: {final_score:.2f}/{max_points}")

# Последняя строка — JSON, который читает harness
final = {
    "name": full_name,
    "group": student_group,
    "assignment": assignment_id,
    "score": float(final_score)
}
print(json.dumps(final, ensure_ascii=False))
